In [1]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from Metrics import * 
from TimewiseKfold import *

In [2]:
folds, PROVINCES_NUM = splitting_data(region=2)

In [3]:
#how much each tree contributes to the final prediction
learning_rate = [0.001, 0.01, 0.05, 0.1, 0.2]
#depth of every tree
max_depth = [3, 5, 6, 7, 9]
#when a node in the will split
# gamma = []
#the percentage of data that is sampled randomly to grow each tree
subsample = [0.5, 0.7, 1]
#the percentage of features that will be sampled at random for growing each tree
colsample_bytree = []
#number of boosting rounds 
n_estimators = [100, 400, 800]
#the minimum amound of data requried to create a new node
# min_child_weight = []
#the balance of postive and negative weights
# scale_pos_weight = []

grid = {
    'learning_rate': learning_rate,
    'max_depth': max_depth,
    'subsample': subsample,
    'n_estimators': n_estimators
}

In [4]:
table = metrics_table()

tscv = TimeSeriesSplit(n_splits=2, test_size=PROVINCES_NUM)

for (i, fold) in enumerate(folds):
    X_train, X_test, y_train, y_test = split_train_test(fold=fold)
    
    xgb_model = XGBRegressor()
    
    grid_search = GridSearchCV(estimator=xgb_model, param_grid=grid, cv=tscv, n_jobs=-1, scoring='r2')
    
    grid_search.fit(X_train, y_train)
    
    row = [MAXIMUM_YEAR - FOLD_NUM + 1 + i]
    
    row += metrics_calculation(y_actual=y_test, y_pred=grid_search.predict(X_test), y_train=y_train)
    
    table.loc[len(table)] = row
    
    del xgb_model
    del grid_search

table  

,Year,RMSE,R2_Score,MSE,MAE,RAE,RSE
0,2018.0,1.631136,0.099138,2.660603,1.428106,0.996167,0.980128
1,2019.0,1.353223,-0.151494,1.831212,1.268980,1.228452,1.157581
2,2020.0,6.140903,-0.050549,37.710684,3.695586,0.994953,1.472677
